## GitlImage Class Usage Examples
This notebook demonstrates the different ways to use the GitlImage class:

**Quick Navigation:**
- [Setup](#Setup-and-Imports)

## Setup-and-Imports

In [ ]:
import time
import os
import argparse
import cProfile
import pstats
import logging

import numpy as np
import astropy.io.fits as pyfits

from corgihowfsc.utils.howfsc_initialization import get_args_jupyter, load_files
from corgihowfsc.sensing.GettingProbes import DefaultProbes

import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

from howfsc.scripts.gitlframes import sim_gitlframe

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')

import matplotlib.pyplot as plt
from corgihowfsc.utils.corgisim_gitl_frames import *

#First, import the module:
import roman_preflight_proper
### Then, run the following command to copy the default prescription file 
# roman_preflight_proper.copy_here()

In [ ]:
from matplotlib.colors import LogNorm

def plot_cgi_corgi(cgi_image,
                   corgi_image,
                   title_cgi='CGI Image',
                   title_corgi='CORGI Image',
                   figsize=(12, 6),
                   log_norm=False,
                   cmap_cgi=None,
                   cmap_corgi=None,
                   colorbar_fraction=0.046,
                   colorbar_pad=0.04,
                   show=True):
    """
    Plot CGI and CORGI images side-by-side. Does not return values, only displays the plot.
    """

    def _safe_log_norm(arr):
        arr = np.asarray(arr)
        pos = arr[np.isfinite(arr) & (arr > 0)]
        vmin = float(pos.min()) if pos.size > 0 else 1e-12
        vmax_val = np.nanmax(arr)
        vmax = float(vmax_val) if np.isfinite(vmax_val) else None
        return LogNorm(vmin=vmin, vmax=vmax) if vmax is not None else LogNorm(vmin=vmin)

    norm0 = _safe_log_norm(cgi_image) if log_norm else None
    norm1 = _safe_log_norm(corgi_image) if log_norm else None

    fig, axs = plt.subplots(1, 2, figsize=figsize)
    im0 = axs[0].imshow(cgi_image, norm=norm0, cmap=cmap_cgi)
    axs[0].set_title(title_cgi)
    im1 = axs[1].imshow(corgi_image, norm=norm1, cmap=cmap_corgi)
    axs[1].set_title(title_corgi)

    fig.colorbar(im0, ax=axs[0], orientation='vertical',
                 fraction=colorbar_fraction, pad=colorbar_pad)
    fig.colorbar(im1, ax=axs[1], orientation='vertical',
                 fraction=colorbar_fraction, pad=colorbar_pad)

    plt.tight_layout()
    if show:
        plt.show()


Loading arguments from files

In [ ]:
# Load arguments
args = get_args_jupyter(jacpath=defjacpath)

In [ ]:
# Initialise variables etc
otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath
    
modelpath, cfgfile, jacfile, cstratfile, probe0file, probe1file, probe2file, hconffile, n2clistfiles = load_files(args, howfscpath)

# cfg
cfg = CoronagraphMode(cfgfile)

# cstrat 
cstrat = ControlStrategy(cstratfile)

# hconffile - needed because of cgihowfsc and corgihowfsc host_star_properties
hconf = loadyaml(hconffile, custom_exception=TypeError)


Getting the probes for probing

In [ ]:
# Initialize default probes class
probes = DefaultProbes('default')

# Get DM lists
probefiles = {}
probefiles[0] = probe0file
probefiles[2] = probe1file
probefiles[1] = probe2file

dm1_list, dm2_list = probes.get_dm_probes(cfg, probefiles,
                      scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])

# Get probe amplitude lists (this step is slow)
plist, other = probes.get_probe_ap(cfg, dm1_list, dm2_list)

## params for get_image 
dm1v = dm1_list[1] # getting probe images 
dm2v = dm2_list[1] # getting probe images 
crop = (0, 0, 500, 500) # dummy values for corgisim

fixedbp = np.zeros((1024,1024), dtype=bool)
peakflux = 4.72154e+06
gain = 5000
exptime = 46.9043

nrow = 153
ncol = 153
lrow = 436
lcol = 436

## Example - calling cgihowfsc-gitl and mapping to corgi optics model

In [ ]:
# Create cgi-howfsc instance
cgihowfsc_img = GitlImage(
    backend='cgi-howfsc',
    cor='narrowfov',
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object  
    hconf=hconf      # Your host config with stellar properties
)

# Initialize with corgisim backend
corig_img = GitlImage(
    cfg=cfg,
    cstrat=cstrat,
    hconf=hconf,
    backend='corgihowfsc',
    cor='narrowfov'
)

cgi_image = cgihowfsc_img.get_image(dm1v, dm2v, exptime, crop=(lrow, lcol, nrow, ncol), peakflux=peakflux)
corgi_image = corig_img.get_image(dm1v, dm2v, exptime)

plot_cgi_corgi(cgi_image, corgi_image)

## Issue: Mapping cgi-howfsc into corgi-howfsc

Here is the situation: 
For cgi-howfsc, we will be working with the same instance, just looping over the wavelength for the gitlImage, or calculating whatever quantities we are looking into 

However for corgi-howfsc, we will run into this situation: 
- Band 1: 1a, 1b, 1c --> 3 instances --> each instance might have 3 - 7 e-field calculations. 